# A4

The Traveling Salesman Problem (TSP)
The TSP tries to answer the following question: "Given a list of cities and the distances
between each pair of cities, what is the shortest possible route that visits each city exactly
once and returns to the origin city?”
The input of the problema is a list of cities and their connections, usually in a graph
format, and the expected output is a list that contains the cities ordered accoring to their
visit order, which is supposed to minimize the total traveling time between them.

## Genetic algorithm

Given the network of cities G, any vector S may be seen as the chromosome
corresponding to the route that travels across all the nodes of G. The objective is the
implementation of a genetic algorithm to obtain the vector that minimizes the total time.

## Imports

In [2]:
%matplotlib inline 

import random
import time
import csv

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pprint import pprint as print 

## Convenience Classes

### City

The City class, which represents a city, possesses the properties of the city and has functions/ methods used for calculating the distance between the city and another city. Each path, represented by a chromosome, is formed by a set of cities.   

In [3]:
class City:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def distance(self, city):
        xDis = abs(self.x - city.x)
        yDis = abs(self.y - city.y)
        distance = np.sqrt((xDis ** 2) + (yDis ** 2))
        return distance
    
    def __repr__(self):
        return "(" + str(self.x) + "," + str(self.y) + ")"

### Fitness

The Fitness class, which represents the fitness function, possesses the properties of a path and has functions/methods used for calculating the fitness value of the path, which is based on the distance of the path. 

In [4]:
class Fitness:
    def __init__(self, route):
        self.route = route
        self.distance = None
        self.fitness = None
    
    def routeDistance(self):
        if self.distance == None:
            pathDistance = 0.0
            for i in range(0, len(self.route)):
                fromCity = self.route[i]
                toCity = None
                if i+1 < len(self.route):
                    toCity = self.route[i+1]
                else:
                    toCity = self.route[0]
                pathDistance += fromCity.distance(toCity)
            self.distance = pathDistance
        return self.distance
    
    def routeFitness(self):
        if self.fitness == None:
        # Fitness function (Simple division) that uses a simple 
        # division that divides one by the distance of the path
            if self.routeDistance() != 0:
                self.fitness = 1 / float(self.routeDistance()) 
                # Note: You must ensure a division by zero does not occur 
        return self.fitness


## Population Initialization  

The population initialization function (or method) performs random initialization. This creates an initial population with completely random chromosomes (or solutions). There are three functions related to population initialization. 

The first function is genCityList() which generates a set of cities from a file.  

In [5]:
def genCityList(filename):
    cityList = []
    
    with open(filename, 'r') as file:
        for line in file:
            _, x, y = line.split()  # Разделение строки на части, игнорирование первого элемента
            cityList.append(City(x=int(x), y=int(y)))
    
    return cityList


The second function is createRoute() which generates a random route (chromosome) from a set of City instances.

In [6]:
def createRoute(cityList):
    route = random.sample(cityList, len(cityList))
    return route

The third function is initialPopulation() which calls the second function repeatedly to create an initial population (a list of routes).

In [7]:
def initialPopulation(popSize, cityList):
    population = []
    for i in range(0, popSize):
        population.append(createRoute(cityList))
    return population

You can run the above functions using the sample runs below. To do so, simply change the cell type from Markdown to Code.

Sample run 1 initializes 12 cities in cityList as follows:

cityList = genCityList('cities10.txt') 
print(cityList)

Sample run 2 initializes 12 cities in cityList and creates a population with three routes as follows:

cityList = genCityList('cities10.txt') 
population = initialPopulation(3, cityList) 
print(population)

## Selection

Parents selection selects chromosomes with high fitness values from a population. Survivor selection selects chromosomes with higher fitness values to form the population of the next generation. The population size is len(population), so we have len(population) in this population. 

### Parent Selection

There are three implementations for parent selection. The first parentSelection() performs random selection.

In [8]:
def parentSelection(population, poolSize=None):
    if poolSize == None:
        poolSize = len(population)
        
    matingPool = []
    
    for i in range(0, poolSize):
        fitness = Fitness(population[i]).routeFitness()
        matingPool.append(random.choice(population))
      
    return matingPool

The second parentSelection() performs Tournament Selection.

In [9]:
def parentSelection(population, poolSize=None):
    if poolSize == None:
        poolSize = len(population)
        
    matingPool = []
    
    matingPool = population[0:poolSize]
    
    return matingPool

The third parentSelection() performs Proportional Selection.

In [10]:
def parentSelection(population, poolSize=None):
    if poolSize == None:
        poolSize = len(population)
        
    matingPool = []

    matingPool = population[0:poolSize]
    
    return matingPool

### Survival Selection

In [11]:
def survivorSelection(population, eliteSize):

    elites = []
    
    for i in range(eliteSize):
        elites.append(population[i])
    
    return elites

You can run the above functions using the sample runs below. To do so, simply change the cell type from Markdown to Code. 

Sample run 1 initializes 12 cities in cityList, creates a population with four routes, and creates a pool of parents as follows:

population = initialPopulation(4, genCityList('cities10.txt'))
matingpool = parentSelection(population, 4) 
print('Initial population') 
print(population) 
print('Mating pool') 
print(matingpool)

Sample run 2 initializes 12 cities in cityList, creates a population with four routes, select an elite chromosome as follows:

population = initialPopulation(4, genCityList('cities10.txt'))
elites = survivorSelection(population, 1)
print('Initial population')
print(population)
print('Selected elites')
print(elites)

## Crossover


In [12]:
def insertGene(insert, insert_index, child1, child2, fixed_child1, fixed_child2):
    # print("(" + str(insert) + ", " + str(insert_index) + ", " + str(child1) + ", " + str(child2) + ")")

    insert_in_child1 = insert in fixed_child1
    if insert_in_child1 == False:
        child1.insert(insert_index, insert)
    else:
        found_index = fixed_child1.index(insert)
        insert = fixed_child2[found_index]
        return insertGene(insert, insert_index, child1, child2, fixed_child1, fixed_child2)

In [13]:
parent1 = [2,1,0,7,3,5,4,6]
parent2 = [6,1,0,5,2,3,4,7]

print(parent1)
print(parent2)

child1 = parent2[2:6] # 2-5
child2 = parent1[2:6] # 2-5

for i in range(0, 2): # 0-1
    insertGene(parent1[i], i, child1, child2, child1, child2)
for i in range(6, 8): # 6-7
    insertGene(parent1[i], i, child1, child2, child1, child2)

for i in range(0, 2): # 0-1
    insertGene(parent2[i], i, child2, child1, child2, child1)
for i in range(6, 8): # 6-7
    insertGene(parent2[i], i, child2, child1, child2, child1)

print(child1)
print(child2)

[2, 1, 0, 7, 3, 5, 4, 6]
[6, 1, 0, 5, 2, 3, 4, 7]
[7, 1, 0, 5, 2, 3, 4, 6]
[6, 1, 0, 7, 3, 5, 4, 2]


In [14]:
import random as rd

def crossover(parent1, parent2):
    first_crossover = rd.randint(0, len(parent1)-1)
    second_crossover = rd.randint(0, len(parent1)-1)
    while second_crossover - first_crossover != 2:
        first_crossover = rd.randint(0, len(parent1)-1)
        second_crossover = rd.randint(0, len(parent1)-1)
    # print("Crossovers: " + str(first_crossover) + ", " + str(second_crossover))
        
    # swap the genes between the crossover
    child2 = parent1[first_crossover : second_crossover+1]
    child1 = parent2[first_crossover : second_crossover+1]
    fixed_child1 = child1.copy()
    fixed_child2 = child2.copy()
    
    # fill in the rest of the genes
    for i in range(0, first_crossover):
        insertGene(parent1[i], i, child1, child2, fixed_child1, fixed_child2)
    for i in range(second_crossover+1, len(parent1)):
        insertGene(parent1[i], i, child1, child2, fixed_child1, fixed_child2)
        
    for i in range(0, first_crossover):
        insertGene(parent2[i], i, child2, child1, fixed_child2, fixed_child1)
    for i in range(second_crossover+1, len(parent1)):
        insertGene(parent2[i], i, child2, child1, fixed_child2, fixed_child1)
    
    return child1, child2

In [15]:
parent1 = [2,1,0,7,3,5,4,6]
parent2 = [6,1,0,5,2,3,4,7]
print(parent1)
print(parent2)
child1, child2 = crossover(parent1, parent2)
print(child1)
print(child2)

[2, 1, 0, 7, 3, 5, 4, 6]
[6, 1, 0, 5, 2, 3, 4, 7]
[6, 1, 0, 7, 3, 5, 4, 2]
[2, 1, 0, 5, 6, 3, 4, 7]


Crossover selects two parents from the mating pool to produce a new generation of the same size.

In [16]:
def breedPopulation(matingpool):
    children = []

    for i in range(1, len(matingpool), 2):
        child1, child2 = crossover(matingpool[i-1], matingpool[i])
        children.append(child1)
        children.append(child2)
    
    return children

In [17]:
parent1 = [2,1,0,7,3,5,4,6]
parent2 = [6,1,0,5,2,3,4,7]
matingpool = [parent1, parent2]

print(matingpool)
children = breedPopulation(matingpool)
print(children)

[[2, 1, 0, 7, 3, 5, 4, 6], [6, 1, 0, 5, 2, 3, 4, 7]]
[[3, 1, 0, 5, 2, 7, 4, 6], [6, 1, 0, 7, 3, 2, 4, 5]]


You can run the above functions using the sample run below. To do so, simply change the cell type from Markdown to Code. The sample run initializes 2 chromosomes in the population, and performs crossover among the two parents. 

population = initialPopulation(2, genCityList('cities10.txt'))
parent1, parent2 = population
child1, child2 = crossover(parent1, parent2)
print('Parents')
print(parent1)
print(parent2)
print('Children')
print(child1)
print(child2)

## Mutation

Mutation mutates a single chromosome to get a mutated chromosome so that genetic algorithm can converge to a shorter path quickly. In the Travelling Saleman Problem, a mutated chromosome must be a valid path. As an example, the shift mutation shifts a single gene in the [1 2 3 4 5 6 7 8 9 10] chromosome to generate the [1 2 4 5 6 7 3 8 9 10] mutated chromosome. As another example, the shift mutation shifts two consecutive genes in the [1 2 3 4 5 6 7 8 9 10] chromosome to generate the [1 4 5 6 7 2 3 8 9 10] mutated chromosome.

In [18]:
def mutate(route, mutationProbability):
    mutated_route = route[:]
    for i in range(0,len(route)):
        if (rd.random() < mutationProbability):

            current_gene = route[i]
            rand_insert_index = rd.randint(0, len(route)-1)            
            mutated_route.remove(current_gene)
            mutated_route.insert(rand_insert_index, current_gene)
    
    return mutated_route

Mutation runs over the entire population and mutates each chromosome in the population with a small mutationProbability. 

In [19]:
def mutation(population, mutationProbability):
    mutatedPopulation = []
    for i in range(0, len(population)):
        mutatedIndividual = mutate(population[i], mutationProbability)
        mutatedPopulation.append(mutatedIndividual)
    return mutatedPopulation

In [20]:
route = [1,2,3,4,5,6,7,8,9,10]
route2 = [10,9,8,7,6,5,4,3,2,1]
population = [route, route2]

mutatedPopulation = mutation(population, 0.1)
print(mutatedPopulation)

[[1, 3, 4, 6, 7, 8, 2, 9, 5, 10], [10, 8, 6, 4, 5, 9, 7, 3, 2, 1]]


You can run the above functions using the sample run below. To do so, simply change the cell type from Markdown to Code. The sample run initializes a route comprised of 12 cities in cityList, and then mutates it as follows:

route = genCityList('cities10.txt')
mutated = mutate(route, 1)  # Give a pretty high chance for mutation
print('Original route')
print(route)
print('Mutated route')
print(mutated)

## Running One Generation (or Interation)

Here, we run one generation of genetic algorithm. 

In [21]:
def oneGeneration(population, eliteSize, mutationProbability):
    
    # First we preserve the elites
    elites = survivorSelection(population, eliteSize)
    
    # Then we calculate what our mating pool size should be and generate
    # the mating pool
    poolSize = len(population) - eliteSize
    matingpool = parentSelection(population, poolSize)
        
    # Then we perform crossover on the mating pool
    children = breedPopulation(matingpool)
    
    # We combine the elites and children into one population
    new_population = elites + children
    
    # We mutate the population
    mutated_population = mutation(new_population, mutationProbability)
        
    return mutated_population

You can run the above functions using the sample run below. To do so, simply change the cell type from Markdown to Code. The sample run initializes a population comprised of 5 chromosomes based on 12 cities in cityList, and then run one generation (or iteration) of genetic algorithm as follows:

## Running Many Generations (or Interations) 

In [28]:
filename = 'st70.txt'
popSize = 20
eliteSize = 5
mutationProbability = 0.01
iteration_limit = 1000

cityList = genCityList(filename)

population = initialPopulation(popSize, cityList)
distances = [Fitness(p).routeDistance() for p in population]
min_dist = min(distances)
print("Best distance for initial population: " + str(min_dist))

for i in range(iteration_limit):
    population = oneGeneration(population, eliteSize, mutationProbability)
    distances = [Fitness(p).routeDistance() for p in population]
    index = np.argmin(distances)
    best_route = population[index]
    min_dist = min(distances)
    print("Best distance for population in iteration " + str(i) +
          ": " + str(min_dist))

print("Optimal path is " + str(best_route)) 

'Best distance for initial population: 3124.619570703471'
'Best distance for population in iteration 0: 3134.9752539104006'
'Best distance for population in iteration 1: 3141.68292230221'
'Best distance for population in iteration 2: 3194.967603901494'
'Best distance for population in iteration 3: 3194.967603901494'
'Best distance for population in iteration 4: 3162.9001344014073'
'Best distance for population in iteration 5: 3162.9001344014073'
'Best distance for population in iteration 6: 3072.4184725335945'
'Best distance for population in iteration 7: 3162.9001344014073'
'Best distance for population in iteration 8: 3168.1400984851566'
'Best distance for population in iteration 9: 3064.153467322064'
'Best distance for population in iteration 10: 3264.8431606843355'
'Best distance for population in iteration 11: 3238.1691702121607'
'Best distance for population in iteration 12: 3256.4147912824833'
'Best distance for population in iteration 13: 3346.9165240414345'
'Best distance for 